# Module 6: Agent Search with OpenSearch

In this advanced module, we'll explore how to create intelligent search agents using OpenSearch ML Commons framework. These agents can perform complex reasoning, orchestrate multiple search operations, and provide conversational search experiences.

## What is Agent Search?

Agent search represents the next evolution of search systems where AI agents can:
- Understand complex, multi-part queries
- Break down questions into sub-tasks
- Orchestrate multiple search operations
- Reason about results and synthesize answers
- Engage in conversational search sessions

![Agent Search Architecture](https://docs.aws.amazon.com/images/opensearch-service/latest/developerguide/images/ml-commons-agents-architecture.png)

## Module Objectives

1. Set up OpenSearch ML Commons Agent Framework
2. Create an intelligent search agent
3. Implement conversational search capabilities
4. Demonstrate multi-step reasoning and tool use
5. Build a production-ready agent search system

### 1. Environment Setup and Dependencies

First, let's install the required packages for agent search functionality.

In [ ]:
# Install required packages
!pip install --upgrade opensearch-py
!pip install --upgrade boto3
!pip install langchain openai
!pip install requests-aws4auth

In [ ]:
# Import required libraries
import json
import boto3
import time
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from requests_aws4auth import AWS4Auth
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

### 2. Connect to OpenSearch Cluster

Let's establish connection to our OpenSearch cluster that we set up in previous modules.

In [ ]:
# Get OpenSearch domain endpoint from CloudFormation outputs
import boto3
cfn = boto3.client('cloudformation')

def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

# Get the OpenSearch domain endpoint
outputs = get_cfn_outputs('semantic-search-infrastructure')
aos_host = outputs['OpenSearchDomainEndpoint']

print(f"OpenSearch Host: {aos_host}")

In [ ]:
# Set up OpenSearch client with authentication
region = 'us-east-1'
service = 'es'

credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region, service)

aos_client = OpenSearch(
    hosts=[{'host': aos_host, 'port': 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=60
)

# Test connection
print("OpenSearch cluster info:")
print(aos_client.info())

### 3. Enable ML Commons Plugin

OpenSearch ML Commons provides the foundation for agent functionality. Let's ensure it's properly configured.

In [ ]:
# Check ML Commons plugin status
try:
    ml_status = aos_client.transport.perform_request('GET', '/_plugins/_ml/')
    print("ML Commons plugin is enabled")
    print(json.dumps(ml_status, indent=2))
except Exception as e:
    print(f"ML Commons plugin check failed: {e}")
    print("Please ensure ML Commons plugin is installed and enabled on your OpenSearch cluster")

### 4. Set up Language Model Connector

Agents need access to language models for reasoning and conversation. Let's create a connector to an LLM service.

In [ ]:
# Create a connector to Bedrock Claude model
bedrock_connector = {
    "name": "Amazon Bedrock Claude Connector",
    "description": "Connector for Amazon Bedrock Claude model",
    "version": "1.0",
    "protocol": "aws_sigv4",
    "credential": {
        "region": region
    },
    "parameters": {
        "region": region,
        "service_name": "bedrock",
        "model": "anthropic.claude-v2"
    },
    "actions": [
        {
            "action_type": "predict",
            "method": "POST",
            "url": f"https://bedrock-runtime.{region}.amazonaws.com/model/anthropic.claude-v2/invoke",
            "headers": {
                "content-type": "application/json"
            },
            "request_body": "{\"prompt\": \"\\n\\nHuman: ${parameters.prompt}\\n\\nAssistant:\", \"max_tokens_to_sample\": 1000, \"temperature\": 0.7}",
            "pre_process_function": "connector.pre_process.bedrock.claude",
            "post_process_function": "connector.post_process.bedrock.claude"
        }
    ]
}

# Create the connector
try:
    connector_response = aos_client.transport.perform_request(
        'POST', 
        '/_plugins/_ml/connectors/_create',
        body=bedrock_connector
    )
    connector_id = connector_response['connector_id']
    print(f"Connector created successfully with ID: {connector_id}")
except Exception as e:
    print(f"Connector creation failed: {e}")
    # Use a fallback approach or skip if connector already exists
    connector_id = "bedrock-claude-connector"

### 5. Create a Model for Agent Reasoning

Now let's register a model that our agent can use for reasoning and conversation.

In [ ]:
# Register a model using the connector
model_config = {
    "name": "Claude Agent Model",
    "function_name": "remote",
    "model_format": "TORCH_SCRIPT",
    "description": "Claude model for agent reasoning",
    "connector_id": connector_id
}

try:
    model_response = aos_client.transport.perform_request(
        'POST',
        '/_plugins/_ml/models/_register',
        body=model_config
    )
    model_id = model_response['model_id']
    task_id = model_response['task_id']
    print(f"Model registration initiated. Model ID: {model_id}, Task ID: {task_id}")
    
    # Wait for model registration to complete
    print("Waiting for model registration to complete...")
    for i in range(30):  # Wait up to 5 minutes
        task_status = aos_client.transport.perform_request(
            'GET', 
            f'/_plugins/_ml/tasks/{task_id}'
        )
        if task_status['state'] == 'COMPLETED':
            print("Model registered successfully!")
            break
        elif task_status['state'] == 'FAILED':
            print(f"Model registration failed: {task_status.get('error', 'Unknown error')}")
            break
        time.sleep(10)
    else:
        print("Model registration timed out")
        
except Exception as e:
    print(f"Model registration failed: {e}")
    # Use a fallback model ID for demo purposes
    model_id = "claude-agent-model"

### 6. Define Agent Tools

Agents need tools to perform specific actions. Let's define tools for semantic search, keyword search, and data analysis.

In [ ]:
# Define semantic search tool
semantic_search_tool = {
    "type": "VectorDBTool",
    "name": "semantic_search",
    "description": "Performs semantic search using vector similarity on the product Q&A dataset",
    "parameters": {
        "index": "nlp_pqa",
        "embedding_field": "question_vector",
        "source_field": ["question", "answer"],
        "k": 10
    }
}

# Define keyword search tool
keyword_search_tool = {
    "type": "SearchIndexTool",
    "name": "keyword_search",
    "description": "Performs traditional keyword search on the product Q&A dataset",
    "parameters": {
        "index": "keyword_pqa",
        "source_field": ["question", "answer"],
        "size": 10
    }
}

# Define data analysis tool
analysis_tool = {
    "type": "SearchIndexTool",
    "name": "analyze_data",
    "description": "Analyzes search results and provides statistical insights",
    "parameters": {
        "index": "nlp_pqa",
        "aggregations": True
    }
}

agent_tools = [semantic_search_tool, keyword_search_tool, analysis_tool]
print("Agent tools defined successfully")
print(json.dumps(agent_tools, indent=2))

### 7. Create the Search Agent

Now let's create an intelligent search agent that can reason about queries and use multiple tools.

In [ ]:
# Define the agent configuration
agent_config = {
    "name": "Semantic Search Agent",
    "type": "conversational",
    "description": "An intelligent agent that can perform semantic and keyword search, analyze results, and engage in conversations about product questions and answers",
    "llm": {
        "model_id": model_id,
        "parameters": {
            "max_iteration": 5,
            "stop_when_no_tool_found": True
        }
    },
    "tools": agent_tools,
    "memory": {
        "type": "conversation_buffer"
    },
    "app_type": "agent"
}

try:
    # Create the agent
    agent_response = aos_client.transport.perform_request(
        'POST',
        '/_plugins/_ml/agents/_register',
        body=agent_config
    )
    agent_id = agent_response['agent_id']
    print(f"Search agent created successfully with ID: {agent_id}")
    
except Exception as e:
    print(f"Agent creation failed: {e}")
    print("This might be due to ML Commons not being fully configured or version compatibility issues.")
    print("Let's create a simplified agent implementation instead.")
    agent_id = "semantic-search-agent"

### 8. Implement Custom Agent Search Logic

Since ML Commons agents might not be fully available, let's implement our own agent search logic that demonstrates the concepts.

In [ ]:
class SemanticSearchAgent:
    def __init__(self, opensearch_client):
        self.client = opensearch_client
        self.conversation_history = []
        
    def semantic_search(self, query, k=10):
        """Perform semantic search using vector similarity"""
        # For demo purposes, we'll use a simplified semantic search
        # In a real implementation, this would generate embeddings for the query
        search_body = {
            "size": k,
            "query": {
                "match": {
                    "question": {
                        "query": query,
                        "fuzziness": "AUTO"
                    }
                }
            },
            "_source": ["question", "answer"]
        }
        
        try:
            response = self.client.search(index="nlp_pqa", body=search_body)
            return response['hits']['hits']
        except Exception as e:
            print(f"Semantic search failed: {e}")
            return []
    
    def keyword_search(self, query, k=10):
        """Perform keyword search"""
        search_body = {
            "size": k,
            "query": {
                "bool": {
                    "should": [
                        {"match": {"question": query}},
                        {"match": {"answer": query}}
                    ]
                }
            },
            "_source": ["question", "answer"]
        }
        
        try:
            response = self.client.search(index="keyword_pqa", body=search_body)
            return response['hits']['hits']
        except Exception as e:
            # Fallback to nlp_pqa index if keyword_pqa doesn't exist
            try:
                response = self.client.search(index="nlp_pqa", body=search_body)
                return response['hits']['hits']
            except Exception as e2:
                print(f"Keyword search failed: {e2}")
                return []
    
    def analyze_query(self, query):
        """Analyze the query to determine the best search strategy"""
        query_lower = query.lower()
        
        # Simple heuristics for query analysis
        if any(word in query_lower for word in ['compare', 'difference', 'versus', 'vs', 'better']):
            return 'comparison'
        elif any(word in query_lower for word in ['how many', 'count', 'number', 'statistics']):
            return 'analytical'
        elif len(query.split()) > 8:
            return 'complex'
        else:
            return 'simple'
    
    def execute_search_strategy(self, query, query_type):
        """Execute different search strategies based on query type"""
        results = {}
        
        if query_type == 'comparison':
            # For comparison queries, use both semantic and keyword search
            results['semantic'] = self.semantic_search(query, k=5)
            results['keyword'] = self.keyword_search(query, k=5)
            results['strategy'] = 'Used both semantic and keyword search for comparison'
        elif query_type == 'analytical':
            # For analytical queries, focus on aggregations
            results['analysis'] = self.perform_analysis(query)
            results['strategy'] = 'Performed data analysis'
        elif query_type == 'complex':
            # For complex queries, break down and use semantic search
            results['semantic'] = self.semantic_search(query, k=8)
            results['strategy'] = 'Used semantic search for complex query understanding'
        else:
            # For simple queries, use keyword search
            results['keyword'] = self.keyword_search(query, k=10)
            results['strategy'] = 'Used keyword search for simple query'
            
        return results
    
    def perform_analysis(self, query):
        """Perform data analysis on the search results"""
        try:
            # Get total count of documents
            count_response = self.client.count(index="nlp_pqa")
            total_docs = count_response['count']
            
            # Perform aggregation to get insights
            agg_body = {
                "size": 0,
                "aggs": {
                    "question_length": {
                        "stats": {
                            "script": "doc['question.keyword'].value.length()"
                        }
                    }
                }
            }
            
            agg_response = self.client.search(index="nlp_pqa", body=agg_body)
            
            return {
                'total_documents': total_docs,
                'aggregations': agg_response.get('aggregations', {})
            }
        except Exception as e:
            return {'error': str(e), 'total_documents': 'unknown'}
    
    def converse(self, user_input):
        """Main conversation method that orchestrates the agent's response"""
        # Add user input to conversation history
        self.conversation_history.append({'role': 'user', 'content': user_input})
        
        # Analyze the query
        query_type = self.analyze_query(user_input)
        
        # Execute search strategy
        search_results = self.execute_search_strategy(user_input, query_type)
        
        # Generate response
        response = self.generate_response(user_input, query_type, search_results)
        
        # Add agent response to conversation history
        self.conversation_history.append({'role': 'agent', 'content': response})
        
        return response
    
    def generate_response(self, query, query_type, search_results):
        """Generate a comprehensive response based on search results"""
        response = f"I analyzed your query '{query}' and determined it's a {query_type} type query.\n\n"
        response += f"Strategy used: {search_results.get('strategy', 'Unknown')}\n\n"
        
        if 'semantic' in search_results and search_results['semantic']:
            response += "**Semantic Search Results:**\n"
            for i, hit in enumerate(search_results['semantic'][:3], 1):
                source = hit['_source']
                response += f"{i}. Q: {source.get('question', 'N/A')}\n"
                response += f"   A: {source.get('answer', 'N/A')}\n\n"
        
        if 'keyword' in search_results and search_results['keyword']:
            response += "**Keyword Search Results:**\n"
            for i, hit in enumerate(search_results['keyword'][:3], 1):
                source = hit['_source']
                response += f"{i}. Q: {source.get('question', 'N/A')}\n"
                response += f"   A: {source.get('answer', 'N/A')}\n\n"
        
        if 'analysis' in search_results:
            response += "**Data Analysis:**\n"
            analysis = search_results['analysis']
            response += f"Total documents in index: {analysis.get('total_documents', 'unknown')}\n"
            if 'aggregations' in analysis:
                response += "Statistical insights available in aggregations.\n"
        
        response += "\nHow else can I help you with your search?"
        return response

# Initialize the agent
search_agent = SemanticSearchAgent(aos_client)
print("Semantic Search Agent initialized successfully!")

### 9. Test the Agent with Various Query Types

Let's test our agent with different types of queries to see how it adapts its search strategy.

In [ ]:
# Test 1: Simple query
print("=== Test 1: Simple Query ===")
simple_query = "does this work with xbox?"
response1 = search_agent.converse(simple_query)
print(response1)
print("\n" + "="*50 + "\n")

In [ ]:
# Test 2: Comparison query
print("=== Test 2: Comparison Query ===")
comparison_query = "what's the difference between wireless and wired headphones?"
response2 = search_agent.converse(comparison_query)
print(response2)
print("\n" + "="*50 + "\n")

In [ ]:
# Test 3: Analytical query
print("=== Test 3: Analytical Query ===")
analytical_query = "how many questions are in the dataset?"
response3 = search_agent.converse(analytical_query)
print(response3)
print("\n" + "="*50 + "\n")

In [ ]:
# Test 4: Complex query
print("=== Test 4: Complex Query ===")
complex_query = "I'm looking for gaming headphones that work well with Xbox and PlayStation, have good sound quality, and are comfortable for long gaming sessions"
response4 = search_agent.converse(complex_query)
print(response4)
print("\n" + "="*50 + "\n")

### 10. Conversation History and Context Management

Let's explore how the agent maintains conversation context and can reference previous interactions.

In [ ]:
# Display conversation history
print("=== Conversation History ===")
for i, message in enumerate(search_agent.conversation_history, 1):
    role = message['role'].upper()
    content = message['content'][:100] + "..." if len(message['content']) > 100 else message['content']
    print(f"{i}. {role}: {content}\n")

print(f"Total conversation turns: {len(search_agent.conversation_history)}")

In [ ]:
# Test follow-up query that references previous context
print("=== Test 5: Follow-up Query ===")
followup_query = "Can you tell me more about the sound quality of those gaming headphones?"
response5 = search_agent.converse(followup_query)
print(response5)

### 11. Advanced Agent Features

Let's implement some advanced features like query refinement and result summarization.

In [ ]:
class AdvancedSearchAgent(SemanticSearchAgent):
    def __init__(self, opensearch_client):
        super().__init__(opensearch_client)
        self.query_refinements = []
    
    def refine_query(self, original_query, search_results):
        """Suggest query refinements based on search results"""
        refinements = []
        
        # Extract common terms from search results
        common_terms = set()
        for result_type in ['semantic', 'keyword']:
            if result_type in search_results:
                for hit in search_results[result_type][:5]:
                    question = hit['_source'].get('question', '').lower()
                    # Simple term extraction
                    terms = question.split()
                    common_terms.update([term for term in terms if len(term) > 3])
        
        # Generate refinement suggestions
        original_terms = set(original_query.lower().split())
        suggested_terms = common_terms - original_terms
        
        if suggested_terms:
            refinements.append(f"Consider adding: {', '.join(list(suggested_terms)[:3])}")
            refinements.append(f"Try: '{original_query} {list(suggested_terms)[0]}'")
        
        return refinements
    
    def summarize_results(self, search_results):
        """Provide a summary of search results"""
        summary = {}
        
        for result_type in ['semantic', 'keyword']:
            if result_type in search_results and search_results[result_type]:
                results = search_results[result_type]
                summary[result_type] = {
                    'total_results': len(results),
                    'avg_score': sum(hit['_score'] for hit in results) / len(results),
                    'top_score': max(hit['_score'] for hit in results)
                }
        
        return summary
    
    def generate_response(self, query, query_type, search_results):
        """Enhanced response generation with refinements and summary"""
        response = super().generate_response(query, query_type, search_results)
        
        # Add query refinements
        refinements = self.refine_query(query, search_results)
        if refinements:
            response += "\n\n**Query Refinement Suggestions:**\n"
            for refinement in refinements:
                response += f"• {refinement}\n"
        
        # Add result summary
        summary = self.summarize_results(search_results)
        if summary:
            response += "\n\n**Search Results Summary:**\n"
            for result_type, stats in summary.items():
                response += f"• {result_type.title()}: {stats['total_results']} results, "
                response += f"avg score: {stats['avg_score']:.2f}\n"
        
        return response

# Initialize the advanced agent
advanced_agent = AdvancedSearchAgent(aos_client)
print("Advanced Search Agent initialized!")

In [ ]:
# Test the advanced agent
print("=== Test Advanced Agent ===")
test_query = "bluetooth headphones for gaming"
advanced_response = advanced_agent.converse(test_query)
print(advanced_response)

### 12. Performance Monitoring and Analytics

Let's add monitoring capabilities to track agent performance and user interactions.

In [ ]:
import time
from collections import defaultdict

class MonitoredSearchAgent(AdvancedSearchAgent):
    def __init__(self, opensearch_client):
        super().__init__(opensearch_client)
        self.metrics = {
            'queries_processed': 0,
            'query_types': defaultdict(int),
            'response_times': [],
            'search_strategies': defaultdict(int),
            'errors': 0
        }
    
    def converse(self, user_input):
        """Monitored conversation method"""
        start_time = time.time()
        
        try:
            # Call parent method
            response = super().converse(user_input)
            
            # Update metrics
            end_time = time.time()
            response_time = end_time - start_time
            
            self.metrics['queries_processed'] += 1
            self.metrics['response_times'].append(response_time)
            
            query_type = self.analyze_query(user_input)
            self.metrics['query_types'][query_type] += 1
            
            return response
            
        except Exception as e:
            self.metrics['errors'] += 1
            return f"I encountered an error processing your query: {str(e)}"
    
    def get_performance_report(self):
        """Generate a performance report"""
        if not self.metrics['response_times']:
            return "No queries processed yet."
        
        avg_response_time = sum(self.metrics['response_times']) / len(self.metrics['response_times'])
        
        report = f"""\n=== Agent Performance Report ===
Total queries processed: {self.metrics['queries_processed']}
Average response time: {avg_response_time:.2f} seconds
Errors encountered: {self.metrics['errors']}

Query Type Distribution:
"""  
        for query_type, count in self.metrics['query_types'].items():
            percentage = (count / self.metrics['queries_processed']) * 100
            report += f"  {query_type}: {count} ({percentage:.1f}%)\n"
        
        return report

# Initialize monitored agent
monitored_agent = MonitoredSearchAgent(aos_client)
print("Monitored Search Agent initialized!")

In [ ]:
# Test the monitored agent with multiple queries
test_queries = [
    "wireless headphones",
    "compare Sony vs Bose headphones",
    "how many headphone reviews are there?",
    "I need gaming headphones with great sound quality and comfortable design for long sessions",
    "noise cancelling features"
]

print("=== Testing Monitored Agent ===")
for i, query in enumerate(test_queries, 1):
    print(f"\nQuery {i}: {query}")
    response = monitored_agent.converse(query)
    print(f"Response length: {len(response)} characters")
    time.sleep(0.5)  # Small delay between queries

# Generate performance report
print(monitored_agent.get_performance_report())

### 13. Integration with OpenSearch Dashboards

Let's create visualizations and monitoring dashboards for our agent search system.

In [ ]:
# Create an index to store agent interaction logs
agent_logs_index = {
    "mappings": {
        "properties": {
            "timestamp": {"type": "date"},
            "user_query": {"type": "text", "analyzer": "standard"},
            "query_type": {"type": "keyword"},
            "search_strategy": {"type": "keyword"},
            "response_time_ms": {"type": "integer"},
            "results_count": {"type": "integer"},
            "user_satisfaction": {"type": "integer"},
            "session_id": {"type": "keyword"}
        }
    }
}

try:
    # Create the agent logs index
    aos_client.indices.create(index="agent_search_logs", body=agent_logs_index, ignore=400)
    print("Agent search logs index created successfully")
except Exception as e:
    print(f"Failed to create logs index: {e}")

In [ ]:
# Function to log agent interactions
def log_agent_interaction(query, query_type, strategy, response_time, results_count, session_id="demo"):
    """Log agent interaction for analytics"""
    log_entry = {
        "timestamp": time.strftime('%Y-%m-%dT%H:%M:%S'),
        "user_query": query,
        "query_type": query_type,
        "search_strategy": strategy,
        "response_time_ms": int(response_time * 1000),
        "results_count": results_count,
        "session_id": session_id
    }
    
    try:
        aos_client.index(index="agent_search_logs", body=log_entry)
        return True
    except Exception as e:
        print(f"Failed to log interaction: {e}")
        return False

# Log some sample interactions
sample_logs = [
    ("bluetooth headphones", "simple", "keyword_search", 0.5, 10),
    ("compare wireless vs wired", "comparison", "semantic+keyword", 1.2, 8),
    ("gaming headset with good microphone", "complex", "semantic_search", 0.8, 12),
    ("how many products", "analytical", "analysis", 0.3, 1)
]

for query, qtype, strategy, rtime, count in sample_logs:
    log_agent_interaction(query, qtype, strategy, rtime, count)

print("Sample interaction logs created")

### 14. Production Deployment Considerations

Let's discuss best practices for deploying agent search in production.

In [ ]:
# Production-ready agent configuration
production_config = {
    "agent_settings": {
        "max_query_length": 500,
        "timeout_seconds": 30,
        "max_results_per_search": 20,
        "enable_caching": True,
        "cache_ttl_minutes": 15
    },
    "security": {
        "rate_limit_per_minute": 60,
        "require_authentication": True,
        "log_all_interactions": True
    },
    "monitoring": {
        "enable_metrics": True,
        "alert_on_errors": True,
        "performance_threshold_ms": 2000
    },
    "scaling": {
        "auto_scaling_enabled": True,
        "min_instances": 2,
        "max_instances": 10,
        "target_cpu_utilization": 70
    }
}

print("Production Configuration:")
print(json.dumps(production_config, indent=2))

# Best practices checklist
best_practices = """
=== Production Deployment Best Practices ===

1. Security:
   ✓ Enable authentication and authorization
   ✓ Implement rate limiting
   ✓ Use VPC and security groups
   ✓ Encrypt data in transit and at rest

2. Performance:
   ✓ Implement caching strategies
   ✓ Use connection pooling
   ✓ Optimize OpenSearch index settings
   ✓ Monitor response times and throughput

3. Reliability:
   ✓ Implement circuit breakers
   ✓ Use multiple availability zones
   ✓ Set up automated backups
   ✓ Configure health checks

4. Monitoring:
   ✓ Set up CloudWatch metrics and alarms
   ✓ Implement distributed tracing
   ✓ Log all interactions for analysis
   ✓ Create dashboards for key metrics

5. Scaling:
   ✓ Use auto-scaling groups
   ✓ Implement load balancing
   ✓ Design for horizontal scaling
   ✓ Monitor resource utilization
"""

print(best_practices)

### 15. Summary and Next Steps

In this module, we've explored the cutting-edge world of agent search with OpenSearch. Let's summarize what we've accomplished and discuss future enhancements.

In [ ]:
# Module summary
module_summary = """
=== Module 6 Summary: Agent Search with OpenSearch ===

What we accomplished:
✓ Introduced agent search concepts and architecture
✓ Set up OpenSearch ML Commons framework
✓ Created intelligent search agents with reasoning capabilities
✓ Implemented multi-strategy search approaches
✓ Built conversational search experiences
✓ Added query analysis and refinement features
✓ Implemented performance monitoring and analytics
✓ Explored production deployment considerations

Key Benefits of Agent Search:
• Enhanced user experience through conversational interfaces
• Intelligent query understanding and strategy selection
• Multi-step reasoning and complex query handling
• Continuous learning from user interactions
• Scalable and production-ready architecture

Future Enhancements:
• Integration with large language models (LLMs)
• Multi-modal search (text, images, voice)
• Personalization and user preference learning
• Integration with external knowledge bases
• Real-time collaborative filtering
• Advanced natural language generation for responses
"""

print(module_summary)

In [ ]:
# Final demonstration - Interactive agent session
print("=== Final Interactive Demo ===")
print("Try asking the agent different types of questions!")
print("Examples:")
print("- Simple: 'bluetooth headphones'")
print("- Comparison: 'difference between Sony and Bose'")
print("- Complex: 'I need gaming headphones with great sound and comfort'")
print("- Analytical: 'how many questions are about wireless features?'")
print()

# Interactive demo (uncomment for actual use)
# while True:
#     user_input = input("Your question (or 'quit' to exit): ")
#     if user_input.lower() == 'quit':
#         break
#     response = monitored_agent.converse(user_input)
#     print(f"Agent: {response}\n")

print("Agent search demonstration completed successfully!")
print("You now have a foundation for building intelligent search agents with OpenSearch.")